In [15]:
import numpy as np
import scipy.stats as si

from stock_centre import *
hq = Stock_Data_Centre()
from opt_assistant import Opt_Assistant
opt_info = Opt_Assistant()

ts_obj = TuShare()
pro = ts_obj.pro

code = '10000001.XSHG'
date = '2015-03-02'



def get_para_of_greeks(code, date, interest_rate_type='1y',his_vol_len=20):
    
    contract_info = opt_info.get_contract_info(code)
    underlying_symbol = contract_info['underlying_symbol'].iloc[0]
    date = pd.to_datetime(date, format='%Y-%m-%d')
    end_date = date
    #print(end_date)
    start_date = date.replace(year=date.year - 1)
    #print(start_date)
    
    #  标的范围
    index_list = ['000300', '000852', '000016']
    etf_list = ['510050', '510300', '510500', '588000', '588080', '159915', '159901', '159919', '159922']
    #  标的历史价格
    if underlying_symbol[:6] in index_list:
        underlying_symbol_for_stock = underlying_symbol[:6]+'.SH'
        underlying_prc_his = hq.get_hq(code=underlying_symbol_for_stock, start_date=start_date, end_date=end_date, index_data=True)
    elif underlying_symbol[:6] in etf_list:
        if underlying_symbol[:6].startswith('5'):
            underlying_symbol_for_stock = underlying_symbol[:6]+'.SH'      
        elif underlying_symbol[:6].startswith('1'):
            underlying_symbol_for_stock = underlying_symbol[:6]+'.SZ'
        underlying_prc_his_Tushare = pro.fund_daily(ts_code=underlying_symbol_for_stock, start_date=pd.to_datetime(start_date, format='%Y-%m-%d').strftime('%Y%m%d'), end_date=pd.to_datetime(end_date, format='%Y-%m-%d').strftime('%Y%m%d'))
        underlying_prc_his = underlying_prc_his_Tushare.set_index(pd.to_datetime(underlying_prc_his_Tushare['trade_date'], format='%Y%m%d').dt.strftime('%Y-%m-%d'))
        underlying_prc_his.sort_index(inplace=True)
    
    s = underlying_prc_his['close'].iloc[0]
    
    k= contract_info['exercise_price'].iloc[0]
    
    r = pro.shibor(start_date=date.strftime('%Y%m%d'), end_date=date.strftime('%Y%m%d'))[interest_rate_type].iloc[0]
    
    exercise_date = contract_info['exercise_date'].iloc[0]

    T= len(hq.get_trade_date(start_date=date, end_date=exercise_date, only_list=True))
    
    sigma = underlying_prc_his.head(his_vol_len)['close'].pct_change().std()* np.sqrt(252)
    
    if contract_info['contract_type'].iloc[0] == 'CO':
        n=1
    elif contract_info['contract_type'].iloc[0] == 'PO':
        n=-1
    
    return (s,k,r,T,sigma,n)
    
greek_input=get_para_of_greeks(code, date, interest_rate_type='1y',his_vol_len=20)


In [19]:
def d(s, k,r,T, sigma) :
    d1 = (np.log(s / k) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return (d1,d2)

   
def delta(s,k,r,T, sigma,n):
    d1 = d(s,k,r,T,sigma)[0]
    delta = n * si.norm.cdf(n * d1)
    return delta

def gamma ( s , k, r,T, sigma ) :
    d1 = d(s,k,r,T,sigma)[0]
    gamma = si.norm.pdf(d1) / (s * sigma * np.sqrt(T))
    return gamma

def vega(s , k,r ,T, sigma ) :
    d1 = d(s,k,r,T,sigma)[0]
    vega = (s * si.norm.pdf(d1) * np.sqrt(T)) / 100
    return vega

def theta(s , k,r,T,sigma,n):
    d1 = d(s,k,r,T,sigma)[0]
    d2 = d(s,k,r,T,sigma)[1]
    
    theta = (-1 *(s * si.norm.pdf(d1) * sigma) / (2 * np.sgrt(T)) - n * r* k *np.exp(-r*T)*si.norm.cdf(n*d2))/365
    return theta

In [20]:
print(d(*greek_input[:-1]))

(95.1375105170524, 94.23260736515685)
